In [1]:
!pip install pandas scikit-learn PyPDF2 nltk sentence-transformers tqdm

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install PyMuPDF

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install tika

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install pytesseract pdf2image

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install joblib pyarrow

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import pandas as pd
import numpy as np
import os
import re
import fitz  # PyMuPDF
from tika import parser as tika_parser
from tika import detector
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import pytesseract
from pdf2image import convert_from_path
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import joblib


tesseract_install_path = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = tesseract_install_path

try:
    detector.from_buffer(None)
    print("Tika server is starting/running. ✅")
except Exception as e:
    print(f"--- TIKA ERROR ---")
    print(f"Could not start Tika. Did you install Java 11+? Error: {e}")

# --- (Your poppler path variable) ---
poppler_install_path = r"C:\Users\VAMSHI KRISHNA BABU\poppler\poppler-25.07.0\Library\bin"


def extract_text_from_pdf(pdf_path):
    """
    Extracts text using a 3-stage robust fallback method:
    1. PyMuPDF (Fast) -> 2. Tika (Robust) -> 3. OCR (Image-based)
    """
    
    # --- Method 1: Try PyMuPDF (fitz) ---
    try:
        text = ""
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text()
        if text.strip():
            return text
        print(f"INFO: PyMuPDF returned no text for {os.path.basename(pdf_path)}. Trying Tika...")
    except Exception as e_fitz:
        if "no such file" in str(e_fitz):
             print(f"ERROR: File path is too long. Please rename: {pdf_path}")
             return ""
        print(f"INFO: PyMuPDF failed on {os.path.basename(pdf_path)} ({e_fitz}). Trying Tika fallback...")

    # --- Method 2: Fallback to Tika ---
    try:
        parsed = tika_parser.from_file(pdf_path)
        text = parsed.get('content')
        if text and text.strip():
            print(f"SUCCESS: Completed with Tika: {os.path.basename(pdf_path)}")
            return text.strip()
        print(f"INFO: Tika also returned no text for {os.path.basename(pdf_path)}. Trying OCR...")
    except Exception as e_tika:
        print(f"ERROR: Tika failed on {os.path.basename(pdf_path)} ({e_tika}). Trying OCR fallback...")

    # --- Method 3: Fallback to OCR (Tesseract) ---
    try:
        # We use the poppler path we defined earlier
        images = convert_from_path(pdf_path, poppler_path=poppler_install_path) 
        
        ocr_text = ""
        for img in images:
            # Now pytesseract knows where to find the 'tesseract.exe'
            ocr_text += pytesseract.image_to_string(img, lang='eng')
        
        if ocr_text.strip():
            print(f"OCR done: Successfully extracted text from {os.path.basename(pdf_path)}")
            return ocr_text
        
        print(f"WARN: All 3 methods failed to find text in {os.path.basename(pdf_path)}. Skipping.")
        return ""
                
    except Exception as e_ocr:
        # This will now only show if poppler is wrong, or Tesseract has another issue
        print(f"ERROR: OCR failed for {os.path.basename(pdf_path)} ({e_ocr}). Skipping.")
        return ""

def preprocess_text(text):
    """Cleans and tokenizes text."""
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return " ".join(filtered_tokens)

# --- (Keep all your imports and helper functions the same) ---
# (e.g., extract_text_from_pdf, preprocess_text, tesseract_cmd, etc.)


# --- Copy all your functions from Phase 1 here ---
# (imports, tesseract_cmd, poppler_path, extract_text_from_pdf, preprocess_text, etc.)

def build_author_database(dataset_path):
    """
    Builds the database by searching RECURSIVELY for all PDFs
    and returns a list of files that failed to process.
    """
    authors_data = []
    failed_files = []
    
    print("Starting recursive search for PDF files...")
    
    for root, dirs, files in tqdm(os.walk(dataset_path), desc="Scanning Folders"):
        for paper_file in files:
            if paper_file.endswith('.pdf'):
                pdf_path = os.path.join(root, paper_file)
                relative_path = os.path.relpath(root, dataset_path)
                author_name = relative_path.split(os.path.sep)[0]
                
                raw_text = extract_text_from_pdf(pdf_path)
                
                if raw_text:
                    processed_text = preprocess_text(raw_text)
                    authors_data.append({
                        'author': author_name,
                        'paper': paper_file,
                        'processed_text': processed_text,
                        'full_path': pdf_path  # <-- THIS IS THE NEWLY ADDED LINE
                    })
                else:
                    failed_files.append(pdf_path)
                        
    return pd.DataFrame(authors_data), failed_files

# --- RE-RUN THE DATABASE BUILDER ---
DATASET_FOLDER = r"C:\Users\VAMSHI KRISHNA BABU\Applied AI A-2\Dataset"
author_df, failed_papers = build_author_database(DATASET_FOLDER)

print(f"\nSuccessfully built database with {len(author_df)} papers.")
print(f"Failed to process {len(failed_papers)} papers.")
author_df.head()

2025-10-23 13:55:47,772 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


Tika server is starting/running. ✅
Starting recursive search for PDF files...


Scanning Folders: 14it [00:14,  1.44s/it]

MuPDF error: library error: FT_New_Memory_Face(Times-Bold): unknown file format

MuPDF error: library error: FT_New_Memory_Face(Times-Bold): unknown file format



Scanning Folders: 29it [00:30,  1.10it/s]

...

Scanning Folders: 71it [01:36,  1.10it/s]

INFO: PyMuPDF returned no text for Combining Global and Local Surrogate Models.pdf. Trying Tika...
INFO: Tika also returned no text for Combining Global and Local Surrogate Models.pdf. Trying OCR...
OCR done: Successfully extracted text from Combining Global and Local Surrogate Models.pdf
INFO: PyMuPDF returned no text for Meta Lamarckian.pdf. Trying Tika...
INFO: Tika also returned no text for Meta Lamarckian.pdf. Trying OCR...
OCR done: Successfully extracted text from Meta Lamarckian.pdf


Scanning Folders: 72it [02:53,  2.41s/it]


Successfully built database with 637 papers.
Failed to process 0 papers.


,author,paper,processed_text,full_path
0,Amit Saxena,A Review of Clustering Techniques.pdf,review clustering techniques developments amit...,C:\Users\VAMSHI KRISHNA BABU\Applied AI A-2\Da...
1,Amit Saxena,Cardioprotection from ischemia.pdf,molecular cellular biochemistry kluwer academi...,C:\Users\VAMSHI KRISHNA BABU\Applied AI A-2\Da...
2,Amit Saxena,Controlled synthesis.pdf,advances natural sciences nanoscience nanotech...,C:\Users\VAMSHI KRISHNA BABU\Applied AI A-2\Da...
3,Amit Saxena,Cutting Edge.pdf,see discussions stats author profiles publicat...,C:\Users\VAMSHI KRISHNA BABU\Applied AI A-2\Da...
4,Amit Saxena,Endogenous IRAK-M Attenuates.pdf,issue necrosis triggers intense inflammatory r...,C:\Users\VAMSHI KRISHNA BABU\Applied AI A-2\Da...


In [8]:
author_df.shape

(637, 4)

In [9]:
author_df['author'][200]

'Geeta Rani'

In [10]:
author_df['processed_text'][0]

'review clustering techniques developments amit saxena mukesh prasad akshansh gupta neha bharill om prakash patel aruna tiwari meng joo er weiping ding chinteng lin department computer science guru ghasidas vishwavidyalaya bilaspur india centre artificial intelligence university technology sydney sydney australia school computational integrative sciences jawaharlal nehru university new delhi india department computer science engineering indian institute technology indore india school electrical electronic engineering nanyang technological university singapore school computer technology nantong university nantong china abstract paper presents comprehensive study clustering exiting methods developments made various times clustering defined unsupervised learning objects grouped basis similarity inherent among different methods clustering objects hierarchical partitional grid density based model based approaches used methods discussed respective states art applicability measures similarity

## Method A: TF-IDF with Cosine Similarity

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# --- 1. Create the TF-IDF Matrix from your database ---
print("Creating TF-IDF matrix...")

# We use ngrams to capture 2-word phrases (e.g., "machine learning")
# min_df=5: ignore terms that appear in fewer than 5 papers
# max_df=0.8: ignore common terms that appear in > 80% of papers
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=5, ngram_range=(1,2))

# This creates a sparse matrix of TF-IDF features for all 637 papers
tfidf_matrix = tfidf_vectorizer.fit_transform(author_df['processed_text'])

print("TF-IDF matrix created with shape:", tfidf_matrix.shape)


# --- 2. Create the Recommendation Function ---
def recommend_with_tfidf(input_pdf_path, k=5):
    """Recommends top k reviewers for an input PDF using TF-IDF."""
    
    # a. Process the input paper
    input_text = extract_text_from_pdf(input_pdf_path)
    processed_input = preprocess_text(input_text)
    
    # b. Transform the input paper into a TF-IDF vector
    # Use .transform() NOT .fit_transform() - we use the vocabulary from our database
    input_vector = tfidf_vectorizer.transform([processed_input])
    
    # c. Compute cosine similarity against all papers in the database
    similarities = cosine_similarity(input_vector, tfidf_matrix).flatten()
    
    # d. Add these scores to our DataFrame
    author_df['similarity_score'] = similarities
    
    # e. Find the top-scoring paper for each author and rank the authors [cite: 43]
    top_authors = author_df.groupby('author')['similarity_score'].max().sort_values(ascending=False)
    
    return top_authors.head(k)

Creating TF-IDF matrix...
TF-IDF matrix created with shape: (637, 61275)


## Method B: Sentence Transformers (Semantic Similarity)

In [12]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# --- 1. Load the Pre-trained Model ---
# This will be downloaded on the first run. 'all-MiniLM-L6-v2' is a great, fast model.
print("Loading sentence embedding model...")
model = SentenceTransformer('all-MiniLM-L6-v2')


# --- 2. Generate Embeddings for all papers in the database ---
# This is a one-time, heavy computation. We are converting all 637 papers into vectors.
print("Generating embeddings for all papers in the database... (This may take a few minutes)")
paper_embeddings = model.encode(author_df['processed_text'].tolist(), show_progress_bar=True)
print("Embeddings generated with shape:", paper_embeddings.shape)


# --- 3. Create the Recommendation Function ---
def recommend_with_embeddings(input_pdf_path, k=5):
    """Recommends top k reviewers for an input PDF using sentence embeddings."""
    
    # a. Process the input paper
    input_text = extract_text_from_pdf(input_pdf_path)
    processed_input = preprocess_text(input_text)
    
    # b. Generate embedding for the single input paper
    input_embedding = model.encode([processed_input])
    
    # c. Compute cosine similarity against all database embeddings
    similarities = cosine_similarity(input_embedding, paper_embeddings).flatten()
    
    # d. Add these scores to our DataFrame
    author_df['similarity_score'] = similarities
    
    # e. Find the top-scoring paper for each author and rank the authors
    top_authors = author_df.groupby('author')['similarity_score'].max().sort_values(ascending=False)
    
    return top_authors.head(k)

Loading sentence embedding model...
Generating embeddings for all papers in the database... (This may take a few minutes)


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

Embeddings generated with shape: (637, 384)


## TOP K-Recommenders

In [13]:
# --- Pick a Test Paper ---
# Let's just grab the 10th paper in your database as an example.
# (You can change 10 to any other number to test different papers)
test_paper_index = 200
INPUT_PAPER_PATH = author_df.iloc[test_paper_index]['full_path']
print(f"--- Testing with paper: {author_df.iloc[test_paper_index]['paper']} ---")
print(f"Paper Author: {author_df.iloc[test_paper_index]['author']}\n")


# --- Run Method A: TF-IDF ---
top_reviewers_tfidf = recommend_with_tfidf(INPUT_PAPER_PATH, k=5)
print("\n--- Top 5 Reviewers (TF-IDF - Keyword Match) ---")
print(top_reviewers_tfidf)


# --- Run Method B: Embeddings ---
top_reviewers_embeddings = recommend_with_embeddings(INPUT_PAPER_PATH, k=5)
print("\n--- Top 5 Reviewers (Embeddings - Semantic Match) ---")
print(top_reviewers_embeddings)

--- Testing with paper: Survey_of_E-Governance_Systems_with_focus_on_Devel.pdf ---
Paper Author: Geeta Rani


--- Top 5 Reviewers (TF-IDF - Keyword Match) ---
author
Geeta Rani           1.000000
Shikha Gupta         0.164366
Dr. Ashish Jain      0.161370
Krishna Asawa        0.158877
Bharghava Rajaram    0.132669
Name: similarity_score, dtype: float64

--- Top 5 Reviewers (Embeddings - Semantic Match) ---
author
Geeta Rani            1.000000
Pinaki Chakraborty    0.739861
Shikha Gupta          0.716990
K.V. Sambasivarao     0.703460
V. Ravi               0.682987
Name: similarity_score, dtype: float32


## Evaluation and Comparison

In [14]:
import pandas as pd

def compare_methods(input_pdf_path, k=5):
    """
    Runs both TF-IDF and Embeddings models on a single paper
    and prints their recommendations side-by-side for comparison.
    """
    print("="*60)
    
    # --- Get Test Paper Info ---
    # Find the row in our DataFrame matching the input path
    test_paper_info = author_df[author_df['full_path'] == input_pdf_path].iloc[0]
    print(f"TESTING PAPER: {test_paper_info['paper']}")
    print(f"ACTUAL AUTHOR: {test_paper_info['author']}")
    print("="*60)

    # --- 1. Run TF-IDF ---
    tfidf_recs = recommend_with_tfidf(input_pdf_path, k)
    
    # --- 2. Run Embeddings ---
    embedding_recs = recommend_with_embeddings(input_pdf_path, k)
    
    # --- 3. Format Results ---
    # Create a DataFrame for easy viewing
    comparison_df = pd.DataFrame({
        'Rank': [i+1 for i in range(k)],
        'TF-IDF Recommendation': tfidf_recs.index,
        'TF-IDF Score': tfidf_recs.values,
        'Embedding Recommendation': embedding_recs.index,
        'Embedding Score': embedding_recs.values
    })
    
    print(comparison_df.to_string()) # .to_string() prints it nicely
    return comparison_df, test_paper_info['author']

In [15]:
# --- Select a few different test papers by their index ---
test_indices = [50, 150, 300, 450, 600]

# This list will hold our results for the final report
evaluation_results = []

for index in test_indices:
    try:
        INPUT_PAPER_PATH = author_df.iloc[index]['full_path']
        result_df, actual_author = compare_methods(INPUT_PAPER_PATH, k=5)
        
        evaluation_results.append({
            "test_paper": author_df.iloc[index]['paper'],
            "actual_author": actual_author,
            "results": result_df
        })
        print("\n\n") # Add space between reports
        
    except Exception as e:
        print(f"Could not test paper at index {index}: {e}\n\n")

TESTING PAPER: A Novel Method for Summarization.pdf
ACTUAL AUTHOR: Aruna Malapati
   Rank    TF-IDF Recommendation  TF-IDF Score Embedding Recommendation  Embedding Score
0     1           Aruna Malapati      1.000000           Aruna Malapati         1.000000
1     2         Dr.Manpreet Kaur      0.281620            Krishna Asawa         0.761541
2     3            Krishna Asawa      0.159655            Pabitra Mitra         0.740464
3     4             Shikha Gupta      0.123185       Pinaki Chakraborty         0.728672
4     5  Ramalinga Swamy Cheruku      0.122058         Dr.Manpreet Kaur         0.726751



TESTING PAPER: A Review on Swarm Intelligence Techniques.pdf
ACTUAL AUTHOR: Dr. Ashish Jain
   Rank   TF-IDF Recommendation  TF-IDF Score Embedding Recommendation  Embedding Score
0     1         Dr. Ashish Jain      1.000000          Dr. Ashish Jain         1.000000
1     2  Prakash Chandra Sharma      1.000000   Prakash Chandra Sharma         1.000000
2     3        Dr. Shikha

## Reviewer-Reviewer Similarity

In [16]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

print("--- Starting Reviewer-Reviewer Similarity Analysis ---")

# 1. Create an "Author Profile" Vector
# We will average the embeddings of all papers for each author
# to create a single vector (of 384 dimensions) that represents their total expertise.

# We take the paper_embeddings (a numpy array) and give it the author names as an index
print("Creating author profile vectors...")
author_vectors_df = (
    pd.DataFrame(paper_embeddings)
    .set_index(author_df['author'])
    .groupby(level=0)  # Group by the index (author name)
    .mean()            # Average all paper vectors for each author
)

print(f"Created {len(author_vectors_df)} unique author profile vectors.")
print("Example profile vector (head):")
print(author_vectors_df.head())
print("\n" + "="*60 + "\n")


# 2. Calculate the Similarity Matrix
# Now we compare every author's profile vector against every other author's.
print("Calculating author-author similarity matrix...")
author_similarity_matrix = cosine_similarity(author_vectors_df)

# Let's put it in a nice DataFrame for easy reading
# The index and columns will both be the list of author names
author_sim_df = pd.DataFrame(
    author_similarity_matrix,
    index=author_vectors_df.index,
    columns=author_vectors_df.index
)

print("Author-Author Similarity Matrix (Top 5x5):")
print(author_sim_df.iloc[:5, :5])
print("\n" + "="*60 + "\n")


# 3. Create the "Find Similar Reviewers" Function
def get_similar_reviewers(author_name, k=5):
    """Finds the top k most similar reviewers to a given reviewer."""
    
    if author_name not in author_sim_df:
        return f"Error: Author '{author_name}' not found in the database."
    
    # Get the specified author's column (or row) of similarity scores
    similar_scores = author_sim_df[author_name]
    
    # Sort the scores from highest to lowest
    # and drop the first one (which is the author themselves, with a score of 1.0)
    top_similar = similar_scores.sort_values(ascending=False).drop(author_name)
    
    # Return the top k
    return top_similar.head(k)

# --- EXAMPLE USAGE ---
# Let's test it with an author from your previous analysis
test_author_1 = "Aruna Malapati"
similar_to_aruna = get_similar_reviewers(test_author_1)

print(f"--- Top 5 Reviewers Similar to {test_author_1} ---")
print(similar_to_aruna)
print("\n")


# --- ANOTHER EXAMPLE ---
test_author_2 = "Dr. Ashish Jain"
similar_to_ashish = get_similar_reviewers(test_author_2)

print(f"--- Top 5 Reviewers Similar to {test_author_2} ---")
print(similar_to_ashish)

--- Starting Reviewer-Reviewer Similarity Analysis ---
Creating author profile vectors...
Created 71 unique author profile vectors.
Example profile vector (head):
                         0         1         2         3         4    \
author                                                                 
Amit Saxena        -0.010342 -0.054523 -0.016319  0.009369  0.035323   
Amita Jain          0.010456 -0.049848 -0.031692  0.000913 -0.002719   
Animesh Chaturvedi -0.035180 -0.053205 -0.037727 -0.000533 -0.016629   
Ankita Jain        -0.036092  0.000081 -0.020570 -0.048243  0.004191   
Arun Chauhan       -0.032100 -0.066101 -0.009475  0.005755  0.021345   

                         5         6         7         8         9    ...  \
author                                                                ...   
Amit Saxena         0.007031 -0.036039  0.075279 -0.023437  0.019549  ...   
Amita Jain         -0.018261 -0.067177  0.065371 -0.033055  0.006535  ...   
Animesh Chaturvedi -0.04